In [1]:
import plotly 
import chart_studio.plotly as py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe_connected"
import IPython
from IPython.display import IFrame
from IPython.display import HTML, display

## Downloading and combining latest data

In [2]:
%run ./data_collection/00-daily-download.py

In [3]:
%run ./data_collection/01-daily-combine.py

Writing combined data to file data/combined/iclaims.csv
done
Writing covidtracking data to file data/combined/covidtracking.csv
Combining covidtracking and fred, and writing data to file data/combined/combined.csv
done


## Load data

In [4]:
df = pd.read_csv('./data/combined/combined.csv')

## Create Variables

In [5]:
df['text'] = df['state'].astype(str) + '\n' + 'Positive: ' + df['positive'].astype(str) + ', ' + 'Deaths: '+ df['death'].astype(str) + ', ' + 'Negative: '+ df['negative'].astype(str) + ', ' + 'Total Tested: '+ df['total'].astype(str)
df = df[df['state'] !=("GU")]
df = df[df['state'] !=("PR")]
df = df[df['state'] !=("AS")]
df = df[df['state'] !=("MP")]
df = df[df['state'] !=("PRI")]
df = df[df['state'] !=("VIR")]
df = df[df['state'] !=("VI")]

In [6]:
def calculate_num_claims_since_emergency_declaration(x: pd.DataFrame) -> int:    
    declaration_date = x["Emergency Declaration Date"].values[0]
    return x[x.date >= declaration_date].number_of_iclaims.sum()


In [7]:
iclaims_since_emergency_by_state = df.groupby("state").apply(calculate_num_claims_since_emergency_declaration)

In [8]:
df['iclaims_since_emergency'] = df.state.apply(
    lambda x: iclaims_since_emergency_by_state[x] if x in iclaims_since_emergency_by_state else None)

In [9]:
def calculate_num_claims_since_stayathome_order(x: pd.DataFrame) -> int:    
    stayathome_date = x["Stay At Home Order Date"].values[0]
    return x[x.date >= stayathome_date].number_of_iclaims.sum()
iclaims_since_stayathome_by_state = df.groupby("state").apply(calculate_num_claims_since_stayathome_order)

In [10]:
df['iclaims_since_stayathome'] = df.state.apply(lambda x: iclaims_since_stayathome_by_state[x] if x in iclaims_since_stayathome_by_state else None)


In [11]:
df = df.sort_values(by=['date'])
df = df[df["date"]>="2020-01-30"]
df = df[df["death"].isna()==False]
df = df[df["positive"].isna()==False]
df['pct_death_over_positive'] = 100*(df['death']/df['positive'])
df['death_rate'] = df['death']/df['positive']
df['high_pct_death'] = df['pct_death_over_positive'].apply(lambda x: '1%+ death rate' if x >= 1 else "below 1% death rate")
df['pct_positive_out_of_total_tested'] = 100*(df['positive']/df['total'])
df['positive_over_pop'] = (df['positive']/df['POPESTIMATE2019'])*100
df['positive_per_100k'] = (df['positive_over_pop']/100)*100000
df['pct_pop_tested'] = (df['total']/df['POPESTIMATE2019'])*100

## Chloropleth maps

In [12]:
df2 = df[df.date == max(df['date'])]

In [13]:
fig = go.Figure(data=go.Choropleth(
    locations=df2['state'],
    z=df2['positive_per_100k'].astype(float),
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=df2['text'],  # hover text
    marker_line_color='white',  # line markers between states
    colorbar_title="Positive per 100,000 people"
))

fig.update_layout(
    title_text='Positive Covid-19 cases per 100,000 people, by State<br>(Hover for breakdown)',
    geo=dict(
        scope='usa',
        projection=go.layout.geo.Projection(type='albers usa'),
        showlakes=True,  # lakes
        lakecolor='rgb(255, 255, 255)'),
)
fig.show();

In [14]:
#scale = 1000
fig = go.Figure(data=go.Choropleth(
    locations=df2['state'],
    z=df2['iclaims_since_emergency'],
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=df2['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Unemployment Insurance Claims"
))

fig.update_layout(
    title_text='Unemployment Insurance Claims since Emergency Declaration,<br>by State (Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
fig.show();

In [15]:
#scale = 1000
fig = go.Figure(data=go.Choropleth(
    locations=df2['state'],
    z=df2['inIcuCumulative'],
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=df2['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Number of Covid patients in ICU, Cumulative"
))

fig.update_layout(
    title_text='Number of Covid patients in ICU, Cumulative,<br>for ID, MN, WI, OH, VA (Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
fig.show();

In [16]:
#scale = 1000
fig = go.Figure(data=go.Choropleth(
    locations=df2['state'],
    z=df2['onVentilatorCumulative'],
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=df2['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Number of Covid patients on Ventilators, Cumulative"
))

fig.update_layout(
    title_text='Number of Covid patients on Ventilators, Cumulative,<br>for OR, VA, and AR (Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
fig.show();

In [17]:
#scale = 1000
fig = go.Figure(data=go.Choropleth(
    locations=df2['state'],
    z=df2['hospitalizedCumulative'],
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=df2['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Number of Covid patients Hospitalized, Cumulative"
))

fig.update_layout(
    title_text='Number of Covid patients Hospitalized, Cumulative<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
fig.show();

In [18]:
#scale = 1000
fig = go.Figure(data=go.Choropleth(
    locations=df2['state'],
    z=(df2['hospitalizedCumulative']/df2['POPESTIMATE2019'])*100000,
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=df2['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Hospitalization per 100,000 people"
))

fig.update_layout(
    title_text='Hospitalization per 100,000 people<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
fig.show();

In [19]:
#scale = 1000
fig = go.Figure(data=go.Choropleth(
    locations=df2['state'],
    z=(df2['hospitalizedCumulative']/df2['positive'])*100,
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=df2['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Hospitalization rate for positive cases (in %)"
))

fig.update_layout(
    title_text='Hospitalization rate for positive cases<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
fig.show();

In [20]:
#scale = 1000
fig = go.Figure(data=go.Choropleth(
    locations=df2['state'],
    z=df2['pct_death_over_positive'],
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=df2['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Death rate in %"
))

fig.update_layout(
    title_text='Death rate by state<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
fig.show();

In [21]:
#scale = 1000
fig = go.Figure(data=go.Choropleth(
    locations=df2['state'],
    z=df2['pct_pop_tested'],
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=df2['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Percent of state population tested (in %)"
))

fig.update_layout(
    title_text='Percent of State Population tested<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
fig.show();

In [22]:
#scale = 1000
fig = go.Figure(data=go.Choropleth(
    locations=df2['state'],
    z=df2['iclaims_since_stayathome'],
    locationmode='USA-states',
    colorscale='Oranges',
    autocolorscale=False,
    text=df2['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Number of Unemp. Ins. Claims since Stay-at-home Orders"
))

fig.update_layout(
    title_text='Number of Unemp. Ins. Claims since Stay-at-home Orders, by State<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
fig.show();